# AxCoRSMRI sample notebook

Self-Supervised Realistic Through-Plane MRI Super Resolution from Clinical 2D Axial and Coronal Acquisition.

[For more information, please see the GitHub repository.](https://github.com/TechnionComputationalMRILab/AxCorSRMRI)

In [1]:
## Imports
import torch
assert torch.cuda.is_available()

import warnings
warnings.filterwarnings("ignore", category=UserWarning) 

## Data Preprocess

For first time users, please prepare your data accordingly. 

Create a where each case ID has two files - one for axial and one for coronal. The files should be in nifti format (.nii.gz). The files should be named as follows: "case_id_AXIAL.nii.gz" and "case_id_CORONAL.nii.gz".

For example:
```
    - data_folder
        - 1
            - 1_AXIAL.nii.gz
            - 1_CORONAL.nii.gz
        - 2
            - 2_AXIAL.nii.gz
            - 2_CORONAL.nii.gz
        - 3
            - 3_AXIAL.nii.gz
            - 3_CORONAL.nii.gz
        - 4
            - 4_AXIAL.nii.gz
            - 4_CORONAL.nii.gz
        - 5
            - 5_AXIAL.nii.gz
            - 5_CORONAL.nii.gz
```

Note that you need to have write permissions in the `data_folder` directory, as the code will write the preprocessed data in the same directory. The paths require a trailing slash.

In [8]:
# Resample isotropic NIFTI coronal files
from axcorsrmri import resample_cases

path_to_data_files = r"./data/"
resample_cases(path_dir = path_to_data_files)

Processing files: 100%|██████████| 337/337 [06:51<00:00,  1.22s/it]


In [9]:
# # Create DB file with isotropic, coronal and axial files paths
from axcorsrmri import create_database

path_to_data_files = r"./data/"
create_database(
    path_to_data_files,
    train_frac=0.8,
    test_frac=0.1,
    num_folds=1
)

Processing files: 100%|██████████| 337/337 [00:00<00:00, 112592.04it/s]


## Param Initializaion

Here you can define the main parameters for the model training. Please refer to `Parameters_dictionary.txt` for more details of each parameter.

In [10]:
from axcorsrmri import parser_setup

override_args = {
    "path_to_set": r"./data/",
    "path_to_results": r"./results/",
    "amount_of_files":20,
    "batch_size":12,
    "loss":"L2",
    "gpu_device":"0",
    "amount_of_slices":3,
    "title":"Test",
    "total_samples":100,
    "patch_size":48,
    "epochs": 20,
    "lr_g":0.0001,
    "lr_d": 0.0001,
    "scheduler": "const",
    "max_workers_train": 12,
    "max_workers_valid": 30,
    "valid_batch_size": 40,
    "adversarial_weight_I": 0.02,
    "adversarial_weight_E": 0.02,
    "d_optimizer_step_size":40 ,
    "g_optimizer_step_size": 160,
    "val_epoch": 5,
    "image_save_freq_batch": 100,
    "mage_save_freq_epoch": 5,
    "save_tensor":True,
    "save_nifti":True
}

args = parser_setup(override_args)

## Dataset Creation

Initialize the training, validation and test datasets for the model training. In addition, the function creates a new folder in `path_to_results` to save the results and logs for tracking with tensorboard.


In [11]:
from axcorsrmri import initialize_data
dl_train, dl_valid_lr, dl_valid_hr, dl_test_lr, dl_test_hr, result_dir, writer, config = initialize_data(args)

Building and creating ESRT model from scratch.
Loading the training and validation datasets...
Loading the training and validation datasets...
Loaded training and validation datasets successfully.
Finished data preparation and parameters initialization.


## Training, validation, and testing
Initalize the model and model parameters, and then train it on `dl_train` and validate it on `dl_valid_lr` and `dl_valid_hr`. After the training is done, it is tested on `dl_test_lr` and `dl_test_hr`.

In [12]:
from axcorsrmri import training_validation_test
training_validation_test(dl_train, dl_valid_lr, dl_valid_hr, dl_test_lr, dl_test_hr, result_dir, writer, config)

Building model...
Built model successfully.
Defining all optimizer functions...
Defined all optimizer functions successfully.
Defining all optimizer scheduler functions...
Defined all optimizer scheduler functions successfully.
Defining all loss functions...
Defined all loss functions successfully.
Starting model training...


100%|██████████| 20/20 [25:35:14<00:00, 4605.75s/it]   


END training
Total training time -  01:35:14
START testing
Start Validation
Reconstructing LR images...
Saving tensors for ./data/3239335163204_20170712_cor_2D_FIESTA__isotropic.nii.gz file
Saving tensors for ./data/3239335163204_20170712_cor_2D_FIESTA__isotropic.nii.gz file
END testing


## Reconstruct SR Volumes
Apply the SR model on all `isotropic` files in the model.


In [4]:
from axcorsrmri import test_parser_setup

override_args_test = {
    "path_to_set": r"./data/",
    "path_to_results": r"./results/",
    "path_to_trained_model": r"./Test_06_06_2024_11_47/",
    "amount_of_files":20,
    "batch_size":12,
    "gpu_device":"0,1",
    "amount_of_slices":3,
    "title":"Test",
    "save_tensor":True,
    "save_nifti":True,
}

test_args = test_parser_setup(override_args_test)

In [5]:
from axcorsrmri import reconstruct_SR_volumes_in_folder

reconstruct_SR_volumes_in_folder(test_args)

Creating the dataset...


: 